## Трансформеры

Данные - https://drive.google.com/file/d/1te0QzJB6SNppduuMVcTcxncqudMxRaPi/view?usp=sharing

In [26]:
import torch
from torch import nn
import numpy as np
import time
import tqdm
import sys
import random
import math

random.seed(123)
np.random.seed(123)

Модель будет предсказывать следующую букву, которая должна идти в последовательности. Такие языковые модели применяются в распозновании речи, так как предоставляют дополнительную информацию акустической модели при выборе следующего символа.

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
path = '/content/drive/MyDrive/small_corp_for_test.txt'
file = open(path, 'r')
data = file.readlines()
file.close()
len(data)

700000

In [9]:
sum_of_letters = np.sum(list(map(lambda x: len(x), data))) # всего символов 
sum_of_letters

30576062

In [39]:
joined_text = ''.join(data)

chars = tuple(set(joined_text)) 

print(chars, len(chars)) # вывожу символы и их количество

('ц', 'ь', 'р', 'о', '[', 'ч', 'б', 'ё', 'п', 'ю', 'е', 'м', 'н', 'в', 'й', 'ы', 'я', 'у', 'ж', 'ъ', 'щ', 'ш', ']', 'ф', 'д', 'г', 'к', 'с', 'л', 'т', 'э', 'х', 'а', 'и', 'з', '-') 36


In [40]:
data = [item.replace('\n', '') for item in data]
data = [item.replace(' ', '') for item in data]
data[:10]

['[добро]',
 '[кого]',
 '[капитан]',
 '[нет]',
 '[зачем]',
 '[чтопроисходит]',
 '[чтотакое]',
 '[рассказ]',
 '[никому]',
 '[нучто]']

In [41]:
sum_of_letters = np.sum(list(map(lambda x: len(x), data))) # всего символов 
sum_of_letters

26895842

In [42]:
joined_text = ''.join(data)

chars = tuple(set(joined_text)) 

print(chars, len(chars)) # вывожу символы и их количество

('ц', 'ь', 'р', 'о', '[', 'ч', 'б', 'ё', 'п', 'ю', 'е', 'м', 'н', 'в', 'й', 'ы', 'я', 'у', 'ж', 'ъ', 'щ', 'ш', ']', 'ф', 'д', 'г', 'к', 'с', 'л', 'т', 'э', 'х', 'а', 'и', 'з', '-') 36


### Подготовка текста 

In [43]:
class Preprocessor:
    def __init__(self):
        self.alphabet = '_добсркгаупитнезчмфяжлйвцыэь-шхющёъ][ '
        self.token2ind = {}
        self.ind2token = {}
        for i in range(len(self.alphabet)):
            self.token2ind[self.alphabet[i]] = i
            self.ind2token[i] = self.alphabet[i]
        
    
    def preprocess(self, text, window_size):
        
        pad = [0]
        if type(text) == list:
          text = ' '.join(text)
        pad_number = window_size - len(text) + 1 # количество символов pad для текста
        text = text.lower() 
        token_text = (list(map(lambda x: self.token2ind[x], text))) # заменяю все символы в тексте на индексы
        return ([37] + token_text[1:-2] + [36] + pad_number*pad ), ([37] + token_text[2:-1] + [36] + pad_number*pad) # массивы без первой  и полседней букв

In [44]:
data = list(map(lambda x: "[" + x + "]", data)) # дополняю скобками

data[:10]

['[[добро]]',
 '[[кого]]',
 '[[капитан]]',
 '[[нет]]',
 '[[зачем]]',
 '[[чтопроисходит]]',
 '[[чтотакое]]',
 '[[рассказ]]',
 '[[никому]]',
 '[[нучто]]']

### Подготовка текста для обучения

In [45]:
THRESHOLD = 128 # задаю порог

beg_thre = [] 
for text in data:
  if len(text) <= THRESHOLD: 
    beg_thre.append(text)  # добавляю только тексты которые прошли порог

random.shuffle(beg_thre) # перемешиваю данные

train = np.array(beg_thre[:np.round(len(beg_thre) * 0.85).astype(int)]) # определяю тренировочную выборку

test = np.array(beg_thre[np.round(len(beg_thre) * 0.85).astype(int):]) # определяю тестовую выборку

In [46]:
len(train)

586249

In [47]:
len(test)

103456

In [48]:
len(beg_thre)

689705

### Класс Dataset для обработки данных

In [49]:
class TextDataset(torch.utils.data.Dataset):
    
    def __init__(self, x, preproc, win_size = 128):
      self.x = x
      self.preproc = preproc
      self.win_size = win_size
    
    def __len__(self):
    
        return len(self.x)
    
    def __getitem__(self, idx):
        
        x, y = self.preproc.preprocess(self.x[idx], self.win_size) 
        x = torch.LongTensor(x) # отдельно перевожу массив по которому нужно предсказать
        y = torch.LongTensor(y) # отдельно перевожу массив который нужно предсказать
        
        return x, y
        

In [50]:
preproc = Preprocessor()
train_dataset = TextDataset(train, preproc)
test_dataset = TextDataset(test, preproc)

In [51]:
train_dataset[0]

(tensor([37, 36, 16, 12,  2,  3, 25, 15,  8, 10, 11,  4, 27, 36,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0]),
 tensor([37, 16, 12,  2,  3, 25, 15,  8, 10, 11,  4, 27, 35, 36,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         

### Класс модели

Маскирование символов нобходимо для того, чтобы модель лучше обучалась, выбирая случайные буквы. 


In [52]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [53]:
class LanguageModel(nn.Module):
    def __init__(self, vocab_size,
                  nhead = 8,
                  num_layers = 6,
                  dropout = 0.1):
        super(LanguageModel, self).__init__()
        self.emb = nn.Embedding(vocab_size, 32, padding_idx=0)
        self.pe = PositionalEncoding(32, dropout)
        self.transformer_encoder_layer = nn.TransformerEncoderLayer(32, nhead)
        self.transformer_encoder = nn.TransformerEncoder(self.transformer_encoder_layer, num_layers)
        self.decoder = nn.Sequential(nn.Linear(32, 64),
                                     nn.ReLU(),
                                     nn.Linear(64, vocab_size))
        
        #nn.TransformerDecoder(nn.TransformerDecoderLayer(32, nhead), num_layers)
    
    def forward(self, x, src_mask):

        x = self.pe.forward(self.emb(x))
        x = x.transpose(1, 0)
        x = self.transformer_encoder(x, src_mask)
        x = self.decoder(x)
        return x.transpose(1, 0)
    
    def generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

In [54]:
model = LanguageModel(vocab_size = len('_добсркгаупитнезчмфяжлйвцыэь-шхющёъ][ '))

### Класс для обучения и валидации

In [55]:
class Trainer:
    
    def __init__(self, model, train_dataset, test_dataset):
        
        self.model = model
        
        self.train_batch_size = 64
        self.test_batch_size = 64
        
        self.train_dataloader = torch.utils.data.DataLoader(train_dataset, 
                                                            batch_size=self.train_batch_size,
                                                            shuffle=False, 
                                                            num_workers=1)
        self.test_dataloader = torch.utils.data.DataLoader(test_dataset, 
                                                            batch_size=self.test_batch_size,
                                                            shuffle=False, 
                                                            num_workers=1)
        self.train_dataloader_size = len(self.train_dataloader)
        self.test_dataloader_size = len(self.test_dataloader)

        self.device = 'cuda:0'
        self.criterion = nn.CrossEntropyLoss(ignore_index = 0)
        self.optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        
        self.steps_to_print = 1000
        
    def train_one_epoch(self, epoch_number):
        step = 0
        counted_loss = 0
        current_time = time.time()
        it = 0
        
        model = self.model.to(self.device)
        
        for batch in self.train_dataloader:
          
            x, y = batch

            x, y = x.to(self.device), y.to(self.device)
            
            src_mask = self.model.generate_square_subsequent_mask(128).to(self.device)
            predicted = self.model(x, src_mask)
            predicted = torch.reshape(predicted, (predicted.shape[0]*128, 38))
            y = y.flatten()
            
            loss = self.criterion(predicted, y)
            counted_loss = loss
            loss.backward()
            self.optimizer.step()
            self.optimizer.zero_grad()

            step += 1
            it += 1

            if step%self.steps_to_print == 0:
                result = 'Train epoch '+str(epoch_number)+' | '
                result += 'Step '+str(step)+'/'+str(self.train_dataloader_size)+' | '
                result += 'Counted loss '+str(counted_loss)+' | '
                result += 'ppl '+str(math.exp(counted_loss/it))+' | '
                result += 'time '+str(time.time() - current_time) + ' | '
                print(result)
                current_time = time.time()
                counted_loss = 0
                it = 0
  
    def validate_one_epoch(self, epoch_number):
        step = 0
        counted_loss = 0
        current_time = time.time()
        it = 0

        for batch in self.test_dataloader:
            it+=1
            step+=1
            
            x, y = batch
            x, y = x.to(self.device), y.to(self.device)

            src_mask = self.model.generate_square_subsequent_mask(128).to(self.device)
            
            predicted = self.model(x, src_mask)
            predicted = torch.reshape(predicted, (predicted.shape[0]*128, 38))
            y = y.flatten()
            loss = self.criterion(predicted, y)
            counted_loss = loss
      
            if step%(self.steps_to_print//2) == 0:
                result = 'Validate epoch '+str(epoch_number)+' | '
                result += 'Step '+str(step)+'/'+str(self.test_dataloader_size)+' | '
                result += 'Counted loss '+str(counted_loss)+' | '
                result += 'ppl '+str(math.exp(counted_loss/it))+' | '
                result += 'time '+str(time.time() - current_time) + ' | '
                print(result)
                current_time = time.time()
                counted_loss = 0
                it = 0
        
    def train(self, number_of_epochs):
        for epoch in range(0, number_of_epochs):
            self.model.train
            self.train_one_epoch(epoch)
            with torch.no_grad():
                self.model.eval()
                self.validate_one_epoch(epoch)
            print()

### Обучение

In [56]:
model= Trainer(model, train_dataset, test_dataset)

model.train(number_of_epochs = 1)

Train epoch 0 | Step 1000/9161 | Counted loss tensor(2.4876, device='cuda:0', grad_fn=<NllLossBackward0>) | ppl 1.0024907459764503 | time 69.46962141990662 | 
Train epoch 0 | Step 2000/9161 | Counted loss tensor(2.3590, device='cuda:0', grad_fn=<NllLossBackward0>) | ppl 1.0023618264395462 | time 69.27170610427856 | 
Train epoch 0 | Step 3000/9161 | Counted loss tensor(2.1874, device='cuda:0', grad_fn=<NllLossBackward0>) | ppl 1.0021897617955957 | time 69.3695867061615 | 
Train epoch 0 | Step 4000/9161 | Counted loss tensor(2.0980, device='cuda:0', grad_fn=<NllLossBackward0>) | ppl 1.0021002108844317 | time 69.49207878112793 | 
Train epoch 0 | Step 5000/9161 | Counted loss tensor(2.0432, device='cuda:0', grad_fn=<NllLossBackward0>) | ppl 1.0020452653829692 | time 69.05400657653809 | 
Train epoch 0 | Step 6000/9161 | Counted loss tensor(2.0108, device='cuda:0', grad_fn=<NllLossBackward0>) | ppl 1.0020128175924568 | time 69.18538451194763 | 
Train epoch 0 | Step 7000/9161 | Counted loss t